In [9]:
import numpy as np
import pandas as pd

In [10]:
usecols_all_states = ['z4type', 'effdate']
for i in range(2, 11):
    usecols_all_states.append(f'z4type{i}')
    usecols_all_states.append(f'effdate{i}')
for i in range(1, 11):
    usecols_all_states.append(f'fips{i}')

dtype_all_states = {
    'z4type': 'category',
    'effdate': 'float',
    'fips1': 'object',
}
for i in range(2, 11):
    dtype_all_states[f'z4type{i}'] = 'category'
    dtype_all_states[f'effdate{i}'] = 'float'
    dtype_all_states[f'fips{i}'] = 'object'


df_all_states = pd.read_csv(
    "data/all_states.csv", 
    usecols=usecols_all_states,
    dtype=dtype_all_states
)

In [11]:
list_dict_col_names = [
    {
        'z4type': 'z4type',
        'effdate': 'effdate',
        'fips1': 'fips',
    }
]
for i in range(2, 11):
    list_dict_col_names.append(
        {
            f'z4type{i}': 'z4type',
            f'effdate{i}': 'effdate',
            f'fips{i}': 'fips',
        }
    )


list_df_all_areas = [
    df_all_states[list_dict_col_names[i].keys()].rename(
        columns=list_dict_col_names[i]
    ) for i in range(10)
]

df_all_areas = (
    pd.concat(list_df_all_areas)
        .sort_values('effdate', kind='stable')
        .sort_index(kind='stable')
)

In [48]:
list_df_all_moves_dropna = [
    df.dropna(subset='effdate') for df in list_df_all_areas
]

df_all_moves2 = pd.concat(list_df_all_moves_dropna)

df_all_moves2 = df_all_moves2.set_index([df_all_moves2.index, 'effdate'])

mi_all_moves = pd.MultiIndex.from_frame(df_all_areas)

print(mi_all_moves)

mi_all_moves.levshape

MultiIndex([('S', 201703.0, '06037401312'),
            ('H', 201703.0,           nan),
            (nan,      nan,           nan),
            (nan,      nan,           nan),
            (nan,      nan,           nan),
            (nan,      nan,           nan),
            (nan,      nan,           nan),
            (nan,      nan,           nan),
            (nan,      nan,           nan),
            (nan,      nan,           nan),
            ...
            ('S', 201702.0, '06059110110'),
            ('S', 201702.0, '06059110111'),
            (nan,      nan,           nan),
            (nan,      nan,           nan),
            (nan,      nan,           nan),
            (nan,      nan,           nan),
            (nan,      nan,           nan),
            (nan,      nan,           nan),
            (nan,      nan,           nan),
            (nan,      nan,           nan)],
           names=['z4type', 'effdate', 'fips'], length=50000)


(4, 98, 2761)

In [25]:
gr_all_areas_effdate = df_all_areas.groupby([df_all_areas.index, 'effdate'])


z4types = ('H', 'S')

def reduce_effdate(group):
    filtered = group[group['z4type'].isin(z4types)]
    return filtered.iloc[0]['fips'] if not filtered.empty else ""


df_filtered_areas = gr_all_areas_effdate.apply(reduce_effdate).to_frame()
df_filtered_areas = df_filtered_areas.rename(columns={0: 'destfips'})
df_filtered_areas = df_filtered_areas.reset_index('effdate')

In [26]:
df_filtered_areas['origfips'] = df_filtered_areas.groupby(df_filtered_areas.index)['destfips'].shift(fill_value="first record")